# Regression Analysis of Mohs Hardness Data Using Bayesian Parameter Optimization

The Mohs Hardness scale rates the hardness of different minerals. The data used here combine both real data from mineral data and crystal data as well as a dataset generated using a deep learning model. The process that we have employed to predict the Mohs Hardness value uses different machine learning models: Random Forest, XGBoost, Light GMBoost. To optimize the parameters of each of these models, we used two different optimizers, Bayesian and Optuna, and evaluated our outcomes based on the Median Absolute Error.  

The process that we followed included
- Data Cleaning
- Data Transformation
- Creation of Training and Validation Sets
- Machine Learning Preprocessing
- ML Parameter Optimization
- ML Model Validation
- ML Model Prediction

---
## Set up Dependencies

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

random_state = 42

import pprint
import joblib
from time import time
from functools import partial

# Sklearn functions
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import median_absolute_error, make_scorer
from sklearn.model_selection import KFold, cross_validate, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

# Skopt functions for hyperparameter optimization
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Integer

# Optuna for hyperparameter optimization
import optuna

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))





## Load the Data

In [ ]:
df_train=pd.read_csv("/kaggle/input/playground-series-s3e25/train.csv")
crystals=pd.read_csv("/kaggle/input/prediction-of-mohs-hardness-with-machine-learning/jm79zfps6b-1/Artificial_Crystals_Dataset.csv")
minerals=pd.read_csv("/kaggle/input/prediction-of-mohs-hardness-with-machine-learning/jm79zfps6b-1/Mineral_Dataset_Supplementary_Info.csv")

df_test=pd.read_csv("/kaggle/input/playground-series-s3e25/test.csv")

Since the data come from different sources, we need to look at the column naming and dimensions. 

In [ ]:
df_train.columns
crystals.columns
minerals.columns

This includes the 'Unnamed: 0', 'Formula', and 'Crystal Structure' columns that are not inlcuded in the training/test dataframes, and is missing the 'id.' It also labels the 'Hardness' as 'Hardness (Mohs)' which will need to be modified to match.

This also includes the 'Unnamed: 0'and 'Hardness (Mohs)' which will need to be modified to match.

In [ ]:
crystals.drop(['Unnamed: 0','Formula', 'Crystal structure'], axis=1, inplace=True)
crystals.rename(columns={"Hardness (Mohs)": "Hardness"}, inplace=True)

minerals.drop(['Unnamed: 0'], axis=1, inplace=True)
minerals.rename(columns={"Hardness (Mohs)": "Hardness"}, inplace=True)

In [ ]:
crystals.head()

In [ ]:
minerals.head()

In [ ]:
df_train.tail()

The training dataset includes the id in numerical order, which matches the index. When we add the crystals and minerals data sets to the training dataframe, it will be necessary to create new 'id' numbers for each of the additions to the set, following the values at the end of the training data to maintain unique id numbers.

In [ ]:
# Create id numbers for the crystals starting after the end of the ID numbers for the original dataset

crystals.index = [i + 1 for i in range(len(df_train), len(df_train) + len(crystals))] 
train_with_crystals=pd.concat([df_train, crystals])

# Make the id number the same as the index 
train_with_crystals.id = train_with_crystals.index

# Do the same for the minerals now
minerals.index = [i + 1 for i in range(len(train_with_crystals), len(train_with_crystals) + len(minerals))]
train_full=pd.concat([train_with_crystals, minerals])
train_full.id = train_full.index

In [ ]:
train_full.tail()

In [ ]:
df_train.columns == train_full.columns

---

## Data Cleaning

The above now verifies that all of the column names are matching with the original data and the new appended dataset. Since we added new data, it's possible that some of the values are duplicated. We had previously looked for duplicates in the training data (not shown), and there were not. It's also important to note that the ID must be removed, since all ID values are unique. By setting the ID as the index, we will no longer be using the ID as a feature, which is also important in the training, while still maintaining the value needed for identification. 

In [ ]:
# Set the ID numbers as the index, then look for duplicates

train_full.set_index('id', inplace=True)
len(train_full[train_full.duplicated()])

We found 24 of the newly added rows from the crystals and minerals were duplicates with the training set. We will drop the duplicates using drop_duplicates().

In [ ]:
# Drop the duplicates 
train_full.drop_duplicates(inplace=True)
len(train_full[train_full.duplicated()])

Now that there are no remaining duplicate rows, there are other issues we need to look for. As what we are looking at in the columns are features such as molecular weight, number of electrons, and density, none of these can have a value of zero. It is possible to have a 0.0 Covelant Bonding, particularly if the mineral is formed with all ionic bonds. A simple example of this is NaCl - non-iodized table salt. But with that one exception, all other features should be greater than zero, and most should be greater than 1. The lowest molecular weight is 1.008 for Hydrogen, which is a gas, not a mineral. So any values for things like total number of electrons, weights, etc. must be greater than 1. 

At this point, we also have not looked to see if there are nulls present in the data. 

In [ ]:
# Count null values
train_full.isna().sum()

In [ ]:
# Look for problems with minima - all values should be greater than 0, most should be greater than or equal to 1
train_full.min()

The lowest Hardness in the Mohs Hardness scale is 1.0, so this feature seems to have an accurate measure. The rest of the columns have problems. 

In [ ]:
elec_tot = (train_full[train_full.allelectrons_Total == 0])
den_tot = (train_full[train_full.density_Total == 0])
elec_avg = (train_full[train_full.allelectrons_Average == 0])
val_e_avg = (train_full[train_full.val_e_Average == 0])
weight_avg = (train_full[train_full.atomicweight_Average == 0])
ion_avg = (train_full[train_full.ionenergy_Average == 0])
chi_avg = (train_full[train_full.el_neg_chi_Average == 0])
vdw = (train_full[train_full.R_vdw_element_Average == 0])
cov = (train_full[train_full.R_cov_element_Average == 0])
zaratio = (train_full[train_full.zaratio_Average == 0])
den_avg = (train_full[train_full.density_Average == 0])

print([len(elec_tot), len(den_tot), len(elec_avg), len(val_e_avg), len(weight_avg), len(ion_avg), len(chi_avg), len(vdw), len(cov), len(zaratio), len(den_avg)])

The numbers above show the number of 0.0 values in each of the columns. Since there are likely rows that contain multiple 0.0 valued features, we decided to start with the rows with the highest frequency that are impossible to have a 0.0 value. First, we can look at the atomic weight average. 

In [ ]:
weight_avg.head(10)

Even in the first 10, we can see that 8 of the rows have multiple columns with 0.0 values, which were likely nulls that were replaced with zeros. Since we know the atomic weight must be greater than 1, we can filter for this to see what other features are still all set to 0.0. 

In [ ]:
train_full.loc[train_full.atomicweight_Average > 1.0].min()

By isolating the atomic weights above 1 and reexamining the minima, the next physical quality that should always be greater than 1 is the all_electrons total. Again, even a single hydrogen atom has 1 electron, and a mineral MUST have more electrons than hydrogen. The minimum was determined to be 0.001, which is a strange value to say the least.

In [ ]:
train_full.loc[train_full.allelectrons_Total == 0.001]

Somehow the allelectrons_Total yielded a 0.001 value, while the allelectrons_Average is 6.000. THIS MAKES NO SENSE! I am losing faith in the performance of the Deep Learning that was used to create these values, since, based on their IDs, they are not from the added crystal and mineral values. These also each have 4 valence electrons, but VERY different atomic weights and VERY different hardnesses. Anyway, I think we can all agree that the allelectrons_total needs to be greater than 1.

In [ ]:
clean_train = train_full.loc[(train_full.atomicweight_Average > 1.0) & (train_full.allelectrons_Total > 1.0)]
clean_train.min()

Continuing with the examination of spurious values, we should now look at the densities. Both the total and average have 0.0 minima. We should look at both. 

In [ ]:
clean_train[clean_train.density_Total == 0.0]

In [ ]:
len(clean_train[clean_train.density_Average == 0.0])

So only one value from the total density yielded a bunch of zeros. The Average density had 40 values. We can just eliminate the one from density total, since much of the data within this row are 0.0. 

In [ ]:
clean_train = clean_train.loc[clean_train.density_Total > 0.0]
clean_train.min()

...And the suspicious 0.001 returns in density total, so let's look at what's happening there. 

In [ ]:
clean_train.loc[clean_train.density_Total <= 0.01]

Three more... so we can just filter out any under 0.01. Finally, looking at the density Average. 

In [ ]:
len(clean_train[clean_train.density_Average == 0])

In [ ]:
clean_train[clean_train.density_Average == 0]

These 39 rows actually have all data from the other features included. So we're left with two options: 
- Remove the 39 rows to include the density_Average as a feature for the machine learning. 
- Keep the 39 rows and remove density_Average as a feature for the machine learning. 

In [ ]:
39/len(clean_train)

The rows represent 0.356% of the full dataset for training, so unless we do decide to remove the feature for density_Average, it seems like a better decision to remove the 39 rows. 

In [ ]:
clean_train = clean_train.loc[clean_train.density_Total > 0.01]
clean_train = clean_train.loc[clean_train.density_Average > 0.0]
clean_train.min()

There is only 1 remaining feaure with a value equal to zero. It is the covalent bond average, and as mentioned above, there are possible minerals with 0.0 covelant bond energy if all of the bonds in the molecule are ionic. Van der Waals forces would still be present, as these are the forces between non-bonded atomic nuclei. 

In [ ]:
clean_train.loc[clean_train.R_cov_element_Average == 0.0]

Since all of the other features have nonzero numerical values, we will keep this row in the training set.  
Now that the data is clean, we can look at the distributions. 

In [ ]:
sns.set(rc={'figure.figsize': (20,20)})
clean_train.hist()

Immediately, we can see that the total electrons and total density are heavily skewed right, and likely have outliers. The atomic weight, allelectrons average and density average are also skewed right. 

## Data Transformation

The two element_Averages for Covalent and van der Waals are distances, which are proportional to the cubed root of density. Since the distributions for the element_Averages appear to be closer to a normal distribution, we opted to transform the density as well as the total and average electrons with a cubed root function. 

In [ ]:
clean_train[['allelectrons_Total_T', 'allelectrons_Average_T', 'density_Total_T']] = clean_train[['allelectrons_Total', 'allelectrons_Average', 'density_Total']].apply(lambda x: x**(1/3))
clean_train.drop(['allelectrons_Total', 'allelectrons_Average', 'density_Total'], axis=1, inplace=True)

In [ ]:
clean_train.hist()

To simplify processing the original data, we created the following function to perform all of the steps in the cleaning. 


In [ ]:
clean_train.shape

In [ ]:
def rock_cleaner(df): 
    df = df.loc[(df.atomicweight_Average > 1.0)] # Removes all atomic weights that cannot possibly exist
    df = df.loc[(df.allelectrons_Total > 1.0)] # Removes both non-existent values from the low end
    df = df.loc[(df.density_Total > 0.1)] # Removes those values with spurious and low densities
    df = df.loc[df.density_Average > 0.0] # Removes the rows with density average nulls set to zero (36 rows)
    
    df[['allelectrons_Total_T', 'allelectrons_Average_T', 'density_Total_T']] = df[['allelectrons_Total', 'allelectrons_Average', 'density_Total']].apply(lambda x: np.log(x+1))
    df.drop(['allelectrons_Total', 'allelectrons_Average', 'density_Total'], axis=1, inplace=True)
    return df

In [ ]:
# Pull the original data and use the cleaning function

train_clean = rock_cleaner(train_full)

In [ ]:
train_clean.shape

## Creation of Training and Validation Sets


In [ ]:
y=train_clean.Hardness
X=train_clean.drop(['Hardness'], axis=1)

scaler = StandardScaler()
X_scaled = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns, index=X.index)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y,test_size=0.2,
                                                                random_state=random_state)

In [ ]:
y.tail()

In [ ]:
X_scaled.tail()

## Machine Learning Preprocessing

We used a Stratified K Fold for the cross validation, which required stratification of the data. 

In [ ]:
# Stratify the Target
y_stratified = pd.cut(y_train.rank(method='first'), bins=20, labels=False)

# Set up the scoring function
scoring = make_scorer(partial(median_absolute_error), greater_is_better=False)

# Set the validation strategy
skf = StratifiedKFold(n_splits=7
                      , shuffle=True 
                      , random_state=random_state)

cv_strategy = list(skf.split(X_train, y_stratified))

The first model we tested was a Random Forest Regression model, which we would tune the hyperparameters using Bayesian Cross Validation or using Optuna. 

## Random Forest Parameter Optimization 

In [ ]:
# Setting the regressor

reg = RandomForestRegressor(
    random_state=random_state 
    )

# Setting the search space
search_spaces = {'n_estimators': Integer(100, 2000)
                 , 'max_depth': Integer(2, 30)
                 , 'min_samples_split': Integer(2, 10)
                 , 'min_samples_leaf': Integer(1, 10)

            }

# Wrap everything up into the Bayesian Optimizer
opt = BayesSearchCV(estimator=reg
                    , search_spaces=search_spaces
                    , scoring=scoring
                    , cv=cv_strategy
                    , n_iter=120                                    # max number of trials
                    , n_points=1                                    # number of hyperparameter sets eval at same time
                    , n_jobs=1                                      
                    , return_train_score=False
                    , refit=False
                    , optimizer_kwargs={'base_estimator': 'GP'}     # Gaussian optimizeer parameters
                    , random_state=random_state)

In [ ]:
# Run the optimizer 
overdone_control = DeltaYStopper(delta=0.001)                    # We stop if the gain of the optimization becomes too small
time_limit_control = DeadlineStopper(total_time=60)           # Impose Time limit
callbacks=[overdone_control, time_limit_control]


# opt.fit(X_train, y_train, callback=callbacks)

The best parameters were returned: <br>
max_depth: 25<br>
min_samples_leaf: 3<br>
min_samples_split: 7<br>
n_estimators: 1625<br>

In [ ]:
# Copy of the best parameters, so not having to keep running the optimizer
reg = RandomForestRegressor(n_estimators=1625
    , min_samples_split=7
    , min_samples_leaf=3
    , max_depth=25
    , random_state=random_state)
reg.fit(X_train, y_train) 
rf_predictions_final = reg.predict(X_valid)

print("Generated Test Data Predictions using Final Model:\n", 
                                                  rf_predictions_final)

Next, we want to look at the feature importances.

In [ ]:
importance = reg.feature_importances_

feature_importance = pd.DataFrame(data=importance, index=X_train.columns, columns=['importance']).sort_values(ascending=True, by='importance')

feature_importance.plot(kind='barh', figsize=(12, 8), color='blue')

In [ ]:
output = pd.DataFrame({'RF Prediction': rf_predictions_final 
                       , 'Actual Hardness': y_valid})
output.head(10)

Finally, the output needs to be assessed using a median_absolute_error.

In [ ]:
Med_A_e = median_absolute_error(rf_predictions_final, y_valid)
Med_A_e

---

The other hyperparameter tuning method we used utlitize the Optuna package. 

In [ ]:
def objective(trial, x_train, y_train, cv, scoring):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 220, 1000, step=20)
        , "max_depth": trial.suggest_int("max_depth", 4, 19)
        , "random_state": 42
        , "min_samples_split" : trial.suggest_int("min_samples_split", 2, 19)
        , "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 2, 19)
    }

    gr_reg = RandomForestRegressor(**params)
    scores = cross_validate(gr_reg, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1)
    gr_reg.fit(X_train, y_train) 

    y_pred = gr_reg.predict(X_valid)
    
    # Compute RMSLE
    MedAE = median_absolute_error(y_valid,y_pred)

    return MedAE

In [ ]:
# %%time

# # Create study that minimizes
# study = optuna.create_study(direction="minimize")

# # Wrap the objective inside a lambda with the relevant arguments
# kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

# # Pass additional arguments inside another function
# func = lambda trial: objective(trial, X_train, y_train, cv=kf, scoring="neg_mean_squared_log_error")

# # Start optimizing with 100 trials
# study.optimize(func, n_trials=200)

The estimators using Optuna yielded a similar outcome with different estimators: 

max_depth: 19<br>
min_samples_leaf: 2<br>
min_samples_split: 5<br>
n_estimators: 920<br>

In [ ]:
basic_forest_model = RandomForestRegressor(n_estimators=920,
    min_samples_split=5,
    min_samples_leaf=2,
    max_depth=19,
    random_state=random_state)
basic_forest_model.fit(X_train, y_train) 
rf_predictions_final = basic_forest_model.predict(X_valid)

print("Generated Test Data Predictions using Final Model:\n", 
                                                  rf_predictions_final)

In [ ]:
output = pd.DataFrame({'RF Prediction': rf_predictions_final 
                       , 'Actual Hardness': y_valid})
output.head(10)

In [ ]:
Med_A_e = median_absolute_error(rf_predictions_final, y_valid)
Med_A_e

While there wasn't a huge improvement, with the use of the Random Forest Regressor, the tuning with Optuna did yield better hyperparameters. The Bayesian predictors didn't run as many trials as the Optuna, which could have impacted this; however, the callback for the change in y value was set to 0.001. 

---

Additional models we tested: 
- XGBoost Regression
- Light GBM Regressor

With each of these models, we used the same data inputs, yet the results that we found were similar but did not perform as well as the Random Forest Model using either the Bayesian or Optuna Tuning. 